# Notebook de EDA — Datathon (Esqueleto)

Este notebook contém um *esqueleto* de Análise Exploratória de Dados (EDA) para o **Datathon 2024**.

**Objetivo:**
- Prever o risco de **Defasagem Escolar** (`Defas`).

**Dicionário de Variáveis (Base 2024):**
- **Target:** `Defas` (Defasagem)
- **Notas:** `Portug`, `Matem`, `Inglês`
- **Indicadores Acadêmicos:** `INDE 22`, `IPS`, `IEG`, `IDA`, `IPV`, `IAA`, `IAN`
- **Dados Demográficos/Escolares:** `Fase`, `Turma`, `Ano ingresso`, `Instituição de ensino`, `Pedra 22`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
import os

# matplotlib inline para notebooks
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

# Comentários linha a linha:
# - pandas: leitura e manipulação de dados
# - numpy: operações numéricas
# - matplotlib: geração de gráficos (usado em ambiente offline)
# - scatter_matrix: matriz de dispersão para análise rápida


In [ ]:
# Carregamento do dataset
DATA_PATH = '../data/BASE DE DADOS PEDE 2024 - DATATHON.xlsx'  # <<-- atualize para o caminho correto
try:
    df = pd.read_excel(DATA_PATH)
except Exception as e:
    print('Erro ao carregar arquivo:', e)
    df = pd.DataFrame()

# Exibir amostra e shape
display(df.head())
print('Shape:', df.shape)


In [ ]:
# Filtragem de Colunas de Interesse
cols_interest = [
    'Defas', 'Portug', 'Matem', 'Inglês', 
    'INDE 22', 'IPS', 'IEG', 'IDA', 'IPV', 'IAA', 'IAN',
    'Fase', 'Turma', 'Ano ingresso', 'Instituição de ensino', 'Pedra 22'
]

# Verificar quais colunas realmente existem no DF
cols_to_keep = [c for c in cols_interest if c in df.columns]
missing_cols = [c for c in cols_interest if c not in df.columns]

if len(missing_cols) > 0:
    print(f'Colunas não encontradas: {missing_cols}')

df = df[cols_to_keep].copy()

# Tipos, nulos e estatísticas descritivas
if df.shape[0] > 0:
    display(df.info())
    display(df.describe(include='all').T)
else:
    print('DataFrame vazio — carregue os dados e reexecute esta célula.')


In [ ]:
# Padronização dos Nomes das Colunas (snake_case)
if df.shape[0] > 0:
    # Converter para maiúsculas + substituir espaços por underline
    df.columns = [c.strip().upper().replace(' ', '_') for c in df.columns]
    
    print('Novos nomes das colunas:', df.columns.tolist())
    
    # Missingness: percentual por coluna
    missing_pct = df.isnull().mean().sort_values(ascending=False) * 100
    display(missing_pct.head(50))
    top_miss = missing_pct.head(20)
    ax = top_miss.plot.bar()
    ax.set_ylabel('Percentual de valores faltantes (%)')
    ax.set_title('Top 20 colunas com maior missingness')
    plt.tight_layout()
else:
    print('DataFrame vazio — carregue os dados e reexecute esta célula.')


In [ ]:
# Defina a coluna target e analise sua distribuição
TARGET_COL = 'DEFAS'  # <<-- Atualizado para nome padronizado
if TARGET_COL in df.columns:
    vc = df[TARGET_COL].value_counts(dropna=False)
    display(vc)
    display((vc / len(df) * 100).round(2))
    vc.plot.bar(title=f'Distribuição de {TARGET_COL}')
    plt.ylabel('Contagem')
    plt.tight_layout()
else:
    print(f"Coluna target '{TARGET_COL}' não encontrada no DataFrame. Verifique o nome exato da coluna.")


In [ ]:
# Histogramas para variáveis numéricas principais
numeric_cols = ['PORTUG', 'MATEM', 'IPS', 'IEG', 'INDE_22', 'IDA', 'IPV']
numeric_cols = [c for c in numeric_cols if c in df.columns]
if len(numeric_cols) == 0:
    print('Nenhuma das colunas numéricas de exemplo foi encontrada no DataFrame. Atualize a lista numeric_cols.')
else:
    for col in numeric_cols:
        plt.figure()
        df[col].hist(bins=30)
        plt.title(f'Distribuição: {col}')
        plt.xlabel(col)
        plt.ylabel('Frequência')
        plt.tight_layout()


In [ ]:
# Boxplots por classe do target
if TARGET_COL in df.columns and len(numeric_cols) > 0:
    for col in numeric_cols:
        plt.figure()
        df.boxplot(column=col, by=TARGET_COL)
        plt.title(f'{col} por {TARGET_COL}')
        plt.suptitle('')
        plt.xlabel(TARGET_COL)
        plt.ylabel(col)
        plt.tight_layout()
else:
    print('Não há target ou colunas numéricas definidas corretamente.')


In [ ]:
# Correlação entre variáveis numéricas (heatmap simples usando imshow)
num_df = df.select_dtypes(include=[np.number])
if num_df.shape[1] > 1:
    corr = num_df.corr()
    display(corr)
    plt.figure(figsize=(10, 8))
    plt.imshow(corr, interpolation='nearest')
    plt.colorbar()
    ticks = range(len(corr.columns))
    plt.xticks(ticks, corr.columns, rotation=90)
    plt.yticks(ticks, corr.columns)
    plt.title('Matriz de correlação (Pearson) - variáveis numéricas')
    plt.tight_layout()
else:
    print('Não há variáveis numéricas suficientes para calcular correlação.')


In [ ]:
# Scatter matrix para um subconjunto de colunas (amostra para performance)
cols_for_scatter = numeric_cols[:4]
if 'num_df' in globals():
    cols_for_scatter = [c for c in cols_for_scatter if c in num_df.columns]
else:
    cols_for_scatter = [c for c in cols_for_scatter if c in df.columns]

if len(cols_for_scatter) >= 2:
    sample = df[cols_for_scatter].sample(n=min(500, len(df)), random_state=42)
    scatter_matrix(sample, alpha=0.6, figsize=(10, 10), diagonal='kde')
    plt.suptitle('Scatter matrix (amostra)')
    plt.tight_layout()
else:
    print('Não há colunas suficientes para scatter matrix.')


In [ ]:
# Estatísticas agrupadas por colunas categóricas (ex.: FASE, BOLSISTA)
categorical_cols = ['FASE', 'PEDRA_22']
categorical_cols = [c for c in categorical_cols if c in df.columns]

if len(categorical_cols) > 0 and len(numeric_cols) > 0:
    for cat in categorical_cols:
        display(df.groupby(cat)[numeric_cols].agg(['count','mean','std']).T)
else:
    print('Atualize categorical_cols ou verifique se as colunas existem no DataFrame.')


## Plano de tratamento de missing (exemplo)

- Colunas com > 50% missing: considerar exclusão ou investigação do processo de coleta.
- Numéricas com poucos missing: imputação por mediana ou por modelo (KNN/Iterative).
- Categóricas: preencher com 'missing' ou criar categoria separada.
- Documentar decisões no README e no notebook.


In [ ]:
# Exportar uma amostra limpa para testes rápidos
if df.shape[0] > 0:
    df_sample = df.copy().sample(n=min(5000, len(df)), random_state=42)
    for c in df_sample.select_dtypes(include=[np.number]).columns:
        df_sample[c] = df_sample[c].fillna(df_sample[c].median())
    out_path = '../data/eda_sample.csv'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df_sample.to_csv(out_path, index=False)
    print(f'Amostra salva em: {out_path}')
else:
    print('DataFrame vazio — carregue os dados e reexecute esta célula.')


## Próximos passos sugeridos

1. Definição final do target (documentar no README).
2. Engenharia de features baseada nos índices do dicionário (`INDE_*`, `IPS_*`, `IEG_*`).
3. Pipeline reprodutível (src/feature_engineering.py, src/train.py).
4. Testes unitários para transformações de dados.
5. Prova de conceito com modelo simples (RandomForest/LightGBM) e avaliação por subgrupos.
